# Segmenting and Clustering Neighborhoods in Toronto

## Import libraries and packages

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd 

#@ hidden

## indicate the HTML page to read

In [80]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

## convert html using lxml package

In [81]:
soup=BeautifulSoup(source,'lxml')

## Extract data from website 

In [82]:
table = soup.find('table')
rows = table.tbody.tr

## Read data and create panda dataframe (exclude 'Not Assigned' and blanks)

In [83]:
tpd = pd.DataFrame(columns=['Postcode','Borough LIST','Neighbourhood'])

x = 1
Postcode=''
Borough=''
Neighbourhood=''

for rows in table.find_all('td'):
  
    row = rows.text
    if x == 1:
        Postcode=row
        
    if x == 2:
        Borough=row
        
    if x == 3:
        Neighbourhood=row
        
    x = x + 1
    
    if x > 3:
        
        if Borough.strip() != 'Not assigned':
            
            if Neighbourhood.strip() =='Not assigned':
                Neighbourhood=Borough
            tpd = tpd.append({'Postcode': Postcode, 'Borough LIST': Borough, 'Neighbourhood': Neighbourhood}, ignore_index=True)
        
        x=1
        Postcode=''
        Borough=''
        Neighbourhood=''

## Replace '\n' with blank

In [84]:
#tpd.sort_values(by ='Postcode' )
tpd['Neighbourhood'] = tpd['Neighbourhood'].str.replace('\n',' ')

## Group data frame by Postcode, Borough and Neighbourhood and clean data

In [85]:
#tpd.groupby('Postcode')['Neighbourhood'].apply(list)
#tpd.groupby('Postcode').agg(lambda x: tuple(x))

#Group data
tpd_group = tpd.groupby(by='Postcode').agg({'Borough LIST':lambda x: list(x),'Neighbourhood':lambda x: list(x)},as_index=False)

#Create new column with first value from the list
tpd_group['Borough'] = tpd_group["Borough LIST"].str[0]

#Remove old column
tpd_group.drop(['Borough LIST'], axis=1, inplace=True)

# Convert List to a String
tpd_group['Neighbourhood'] = tpd_group.Neighbourhood.apply(', '.join)

#tpd_group


## Retrieve Geospatial data

In [86]:
df_geo = pd.DataFrame(pd.read_csv('https://cocl.us/Geospatial_data'))
df_geo=df_geo.rename(columns = {'Postal Code':'Postcode'})
df_geo.set_index('Postcode',inplace=True)
#df_geo

## Merge data

In [87]:
tpd_group.join(df_geo)

,Neighbourhood,Borough,Latitude,Longitude
Postcode,,,,
M1B,"Rouge , Malvern",Scarborough,43.806686,-79.194353
M1C,"Highland Creek , Rouge Hill , Port Union",Scarborough,43.784535,-79.160497
M1E,"Guildwood , Morningside , West Hill",Scarborough,43.763573,-79.188711
M1G,Woburn,Scarborough,43.770992,-79.216917
M1H,Cedarbrae,Scarborough,43.773136,-79.239476
M1J,Scarborough Village,Scarborough,43.744734,-79.239476
M1K,"East Birchmount Park , Ionview , Kennedy Park",Scarborough,43.727929,-79.262029
M1L,"Clairlea , Golden Mile , Oakridge",Scarborough,43.711112,-79.284577
M1M,"Cliffcrest , Cliffside , Scarborough Village W...",Scarborough,43.716316,-79.239476


## Print the number of rows 

In [88]:
tpd_group.shape

(103, 2)